<a href="https://colab.research.google.com/github/hfenelsoftllc/telecomdata-consumer-edafiu/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project

## Project Setup and Data Cleanup
Clean and preprocess data, handle missing values, and transform data into suitable format for analysis.

### Setup Data

In [ ]:
# Load Shared Data File
import gdown
file_id = '1ydXWMvPAzlZaOufozBYephHczZXUAwcI'
gdown.download(f'https://drive.google.com/uc?id={file_id}', 'Consumer Data - ISM6644 Final Project.csv', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1ydXWMvPAzlZaOufozBYephHczZXUAwcI
To: /content/Consumer Data - ISM6644 Final Project.csv
100%|██████████| 486k/486k [00:00<00:00, 60.1MB/s]


'Consumer Data - ISM6644 Final Project.csv'

In [ ]:
# Imports
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
import plotly.express as px
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

# Load raw data into from csv into a DataFrame
raw_df = pd.read_csv("Consumer Data - ISM6644 Final Project.csv")
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9993 entries, 0 to 9992
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Service               9993 non-null   object 
 1   Age                   9993 non-null   int64  
 2   CustomerSince         9993 non-null   object 
 3   Customer Gender       9993 non-null   object 
 4   Region                9993 non-null   object 
 5   SupportCallsLastYear  9993 non-null   int64  
 6   AverageBill           9990 non-null   float64
 7   ChurnIndicator        9993 non-null   float64
 8   Satisfaction Score    9991 non-null   float64
 9   Unnamed: 9            0 non-null      float64
 10  Unnamed: 10           0 non-null      float64
dtypes: float64(5), int64(2), object(4)
memory usage: 858.9+ KB


In [ ]:
# Drop unnamed columns
unnamed_cols = [col for col in raw_df.columns if "Unnamed" in col]
raw_df.drop(columns=unnamed_cols, inplace=True)
# Rename Columns to follow one naming scheme
raw_df.rename(columns={'Customer Gender': 'CustomerGender', 'Satisfaction Score': 'SatisfactionScore'}, inplace=True)

raw_df.head()

,Service,Age,CustomerSince,CustomerGender,Region,SupportCallsLastYear,AverageBill,ChurnIndicator,SatisfactionScore
0,fiber,29,6/23/2010 10:27,M,North,2,61.0,0.32,5.0
1,4G,74,6/23/2010 10:27,M,North,0,48.0,0.02,3.0
2,phone,34,6/23/2010 10:27,F,Central,0,34.0,0.02,4.0
3,phone,60,6/24/2010 10:27,M,North,5,70.0,0.23,5.0
4,landline,29,6/24/2010 10:27,M,Central,5,40.0,0.09,5.0


### Functions

#### IQR Method Outlier Detection

In [ ]:
# Function to detect outliers in a given column
# Returns a df with a column marking the outliers
def iqr_outlier_detection(df, col):
    #copy the df into a new one
    _df = df.copy()

    Q1 = _df[col].quantile(0.25)  # First quartile (25th percentile)
    Q3 = _df[col].quantile(0.75)  # Third quartile (75th percentile)
    IQR = Q3 - Q1  # Interquartile range

    # Set thresholds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Detect outliers
    outliers = _df[col].apply(lambda x: x < lower_bound or x > upper_bound)
    # Return outliers
    return outliers

#### Pie Chart

In [ ]:
# Function for ploting a pie chart
# Outputs a pie chart
def pie_chart(df, names, title, color):
  fig = px.pie(df, names=names, title=title,
               color_discrete_sequence=color)
  fig.update_traces(textposition='inside', textinfo='percent+label')
  fig.show()

#### Bar Chart

In [ ]:
# Function for ploting a bar chart
# Outputs a bar chart
def bar_chart(df, title, y_label):
  fig = px.bar(df, title=title, labels={"value": y_label})
  fig.update(layout_showlegend=False)
  fig.show()

### Converting Objects to Data Types

In [ ]:
# Datetime is stored as an object, should be converted to datetime.
raw_df['CustomerSince'] = pd.to_datetime(raw_df['CustomerSince'])
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9993 entries, 0 to 9992
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Service               9993 non-null   object        
 1   Age                   9993 non-null   int64         
 2   CustomerSince         9993 non-null   datetime64[ns]
 3   CustomerGender        9993 non-null   object        
 4   Region                9993 non-null   object        
 5   SupportCallsLastYear  9993 non-null   int64         
 6   AverageBill           9990 non-null   float64       
 7   ChurnIndicator        9993 non-null   float64       
 8   SatisfactionScore     9991 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 702.8+ KB


In [ ]:
# Checked that Gender formatting is consistent
raw_df['CustomerGender'].unique()

array(['M', 'F'], dtype=object)

### Handle Missing Data and Outliers

In [ ]:
# New DataFrame to store clean data
df = raw_df.copy()

In [ ]:
# Iterate through raw DataFrame
for column in raw_df.columns:
  # Only work on numeric columns
  if is_numeric_dtype(raw_df[column]):
    # Fill missing values with median
    df.fillna(df[column].median(), inplace=True)
    # Replace outliers with median using IQR method
    df[column] = np.where(iqr_outlier_detection(df, column), df[column].median(), df[column])

#print(df)

In [ ]:
#Graph after removing outliers

### Forming Groups

In [ ]:
# Group by Service
service_group = df.groupby(df["Service"])

In [ ]:
# Group by Gender
gender_group = df.groupby(df["CustomerGender"])

In [ ]:
# Group by Region
region_group = df.groupby(df["Region"])

### Standardize Data

In [ ]:
df[['SupportCallsLastYear_Std', 'SatisfactionScore_Std']] = (df[['SupportCallsLastYear', 'SatisfactionScore']] - df[['SupportCallsLastYear', 'SatisfactionScore']].mean()) / df[['SupportCallsLastYear', 'SatisfactionScore']].std()

## Exploratory Data Analysis and Data Visualization
Understand data distribution, trends and patterns. Summarize the data using descriptive statistics.

### Demographic

#### Distribution by Region


In [ ]:
pie_chart(df, "Region", "Customer Distribution By Region", px.colors.cyclical.Twilight)

**Insights:**

The distribution shows that there is a higher concentration of customers in the central region.

In [ ]:
bar_chart(service_group.size(), "Number of Customers by Service Type", "Number of Customers")

**Insights:**

The distribution shows that there is a higher concentration of customers using 4G and Phones compared to landline and fiber optics.

#### Age By Region


In [ ]:
bar_chart(region_group["Age"].mean(), "Average Age by Region", "Age")

Insights

There is slightly more age concentration on the South compare to North and Central location

#### Service By Age

In [ ]:
bar_chart(service_group["Age"].mean(), "Average Age by Service Type", "Age")

*Insights*

----- Provide insights here -----

#### Customers Age Distribution

In [ ]:
fig = px.histogram(df, x="Age", nbins=20, title="Age And Gender Distribution of Customers", color="CustomerGender")
fig.update_layout(bargap=0.2)
fig.show()

*Insights*

This customer distribution graph shows that the customers are between 25 and 65 years old. The number of male and female customers seems relatively balanced.

### Churn and Satisfaction

#### Support Calls vs Satisfaction Score
scatter plot of support calls vs satisfaction score

In [ ]:
fig = px.scatter(df,
                 x='SupportCallsLastYear',
                 y='SatisfactionScore',
                 title='Support Calls vs. Satisfaction Score per year Trend',
                 labels={'SupportCallsLastYear':'Number of Support Calls Last Year','SatisfactionScore':'Satisfaction Score'},
                 trendline="ols")

fig.update_layout(
    title_font_size=20,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    plot_bgcolor='rgba(0,0,0,0)',
    yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='LightGray'),
    xaxis=dict(showgrid=False)
)

fig.show()

#### Support Calls and Satisfaction Score By Region
bar chart of support calls by region and a bar chart of satisfaction score by region

In [ ]:
fig1 = px.bar(region_group['SupportCallsLastYear'].mean().reset_index(),
              x='Region',
              y='SupportCallsLastYear',
              title='Average Support Calls by Region',
              labels={'SupportCallsLastYear': 'Average Number of Support Calls'})

fig1.update_layout(
    title_x=0.5,
    plot_bgcolor='white',
    width=900,
    height=500
)
fig1.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
fig1.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')

fig1.show()

fig2 = px.bar(df.groupby('Region')['SatisfactionScore'].mean().reset_index(),
              x='Region',
              y='SatisfactionScore',
              title='Average Satisfaction Score by Region',
              labels={'SatisfactionScore': 'Average Satisfaction Score'})

fig2.update_layout(
    title_x=0.5,
    plot_bgcolor='white',
    width=900,
    height=500
)
fig2.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
fig2.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')

fig2.show()

#### Churn by Loyalty
line chart of churn over customer since

In [ ]:
df["CustomerLoyalty"] = (pd.to_datetime("today") - df["CustomerSince"]).dt.days / 365
fig = px.scatter(df, x="CustomerLoyalty", y="ChurnIndicator", title="Churn by Customer Loyalty (Years Since Joining)", labels={"CustomerLoyalty": "Customer Loyalty (Years)", "ChurnIndicator": "Churn Indicator"}, opacity=0.3)
fig.show()

**Insights**

The graph suggests that new customers that have been with the service between 9 and 12 years have a tendency to keep the service. Customers that have been with the service for than 12 years have a tendency to leave.

Churn rate across region

In [ ]:
fig3 = px.bar(df.groupby('Region')['ChurnIndicator'].mean().reset_index(),
              x='Region',
              y='ChurnIndicator',
              title='Churn Rate by Region',
              labels={'ChurnIndicator': 'Churn Rate (%)',
                     'Region': 'Region'})

fig3.update_layout(
    title_x=0.5,
    plot_bgcolor='white',
    width=900,
    height=500,
    yaxis=dict(tickformat='.1%')
)
fig3.show()

Churn rate across service types

In [ ]:
fig4 = px.bar(df.groupby('Service')['ChurnIndicator'].mean().reset_index(),
              x='Service',
              y='ChurnIndicator',
              title='Churn Rate by Service Type',
              labels={'ChurnIndicator': 'Churn Rate (%)',
                     'Service': 'Service Type'})

fig4.update_layout(
    title_x=0.5,
    plot_bgcolor='white',
    width=900,
    height=500,
    yaxis=dict(tickformat='.1%')
)
fig4.show()

**Insights**

This graph may suggest that the older the technology, the more stable it is in terms of customer loyalty. This could suggest that there is more competition in the newer technologies.

#### Churn rate across gender

In [ ]:
fig5 = px.bar(df.groupby('CustomerGender')['ChurnIndicator'].mean().reset_index(),
              x='CustomerGender',
              y='ChurnIndicator',
              title='Churn Rate by Gender',
              labels={'ChurnIndicator': 'Churn Rate (%)',
                     'CustomerGender': 'Gender'})

fig5.update_layout(
    title_x=0.5,
    plot_bgcolor='white',
    width=900,
    height=500,
    yaxis=dict(tickformat='.1%')
)
fig5.show()

#### Churn Rate across age groups

In [ ]:
df['AgeGroup'] = pd.cut(df['Age'],
                       bins=[0, 25, 35, 45, 55, 65, 100],
                       labels=['18-25', '26-35', '36-45', '46-55', '56-65', '65+'])

fig6 = px.bar(df.groupby('AgeGroup')['ChurnIndicator'].mean().reset_index(),
              x='AgeGroup',
              y='ChurnIndicator',
              title='Churn Rate by Age Group',
              labels={'ChurnIndicator': 'Churn Rate (%)',
                     'AgeGroup': 'Age Group'})

fig6.update_layout(
    title_x=0.5,
    plot_bgcolor='white',
    width=900,
    height=500,
    yaxis=dict(tickformat='.1%')
)
fig6.show()

<ipython-input-26-41c8906c8030>:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Conclusion



This data sucks